In [1]:
import torch
import torch.nn as nn
from torch.nn import Embedding

import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import dataset
import test_lstm

2023-03-08 14:30:20.721613: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
lie_trial_path = './processed_lie/' #60 entries
truth_trial_path = './processed_truth/' #61 entries
MU3D_path = './processed/' # 300 entries

In [3]:
# no split by person
X, Y = dataset.preprocessing(truth_trial_path, lie_trial_path, numOfFrames=10)

TEST_RATIO = 0.2

xTrain, xTest = train_test_split(X, test_size=TEST_RATIO, shuffle=False)
yTrain, yTest = train_test_split(Y, test_size=TEST_RATIO, shuffle=False)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
embedding_layer = Embedding(num_embeddings=10, embedding_dim=4)

positions = torch.tensor([0,1,2,3,4,5,6,7,8,9])
embedded_positions = embedding_layer(positions)
embedded_positions

tensor([[-9.5237e-01,  1.5675e-01,  4.2209e-01,  3.1437e-01],
        [-1.9194e+00, -1.1530e+00, -2.5596e-01, -6.6277e-01],
        [ 1.1716e-02,  9.7424e-01, -3.3911e-01, -6.4918e-01],
        [-7.5915e-01,  3.4919e-01, -4.4198e-01,  5.2211e-01],
        [ 7.0474e-01, -1.4141e+00, -9.7262e-01, -1.9992e-01],
        [ 2.3989e-01,  3.0762e-04,  3.2785e-01,  1.0899e+00],
        [ 5.4010e-02, -1.9672e+00, -8.0634e-01, -3.2001e-01],
        [-7.5895e-01, -7.3678e-01, -1.8904e+00,  1.2929e-01],
        [-1.6781e+00,  7.3380e-01, -1.0691e-01,  2.7702e-01],
        [ 2.4964e-01, -1.2619e+00,  1.0165e+00, -1.5233e+00]],
       grad_fn=<EmbeddingBackward0>)

In [58]:
#embedding
def gen_loc_list(max_frames_count, pos_encoder_size, frames_count = 10):
  embedding = nn.Embedding(max_frames_count, pos_encoder_size)
  input = torch.tensor([i for i in range(frames_count)]).clone().detach()
  loc_list = embedding(input)

  return loc_list

def gen_data(data_arr, embedding_arr):

  data_arr = torch.tensor(data_arr).clone().detach()
  list = torch.cat((data_arr, embedding_arr), 1)

  return list

tensor([[ 0.3846,  0.2573, -0.8865,  0.3230,  0.2880,  0.2500,  0.1300,  1.7700,
          0.0000,  0.1600,  0.3792, -0.1752, -0.3183,  1.9539],
        [ 0.3859,  0.2574, -0.8859,  0.3240,  0.2890,  0.1500,  0.0600,  1.7500,
          0.0000,  0.2400,  0.2904,  0.0217, -0.1931, -0.1589],
        [ 0.3864,  0.2583, -0.8854,  0.3250,  0.2900,  0.1400,  0.0700,  1.7300,
          0.0000,  0.2600, -0.4204,  0.4735,  0.3364, -0.8281],
        [ 0.3874,  0.2542, -0.8862,  0.3260,  0.2850,  0.1600,  0.1000,  1.8200,
          0.0000,  0.2100, -0.4550, -0.1587, -0.1849,  0.1962],
        [ 0.3835,  0.2552, -0.8876,  0.3230,  0.2860,  0.1500,  0.1100,  1.9200,
          0.0000,  0.2500,  0.2708, -0.5133,  1.4081, -1.7879],
        [ 0.3830,  0.2547, -0.8879,  0.3230,  0.2850,  0.1200,  0.1100,  1.9700,
          0.0000,  0.1600, -0.0289,  0.8875, -1.3817, -0.5853],
        [ 0.3842,  0.2522, -0.8881,  0.3240,  0.2820,  0.0600,  0.1200,  2.0400,
          0.0000,  0.1100,  1.3679,  0.3769,  0.1

In [62]:
#model

from torch.nn import TransformerEncoder, TransformerEncoderLayer

class TransformerModel(nn.Module):

  def __init__(self, inFeatCount, num_encoder_layers, n_heads = 5, n_hidden = 500, dropout = 0.3, outFeatCount = 2): #Out = [prob for 0, prob for 1]
    super(TransformerModel, self).__init__()

    encoder_layer = TransformerEncoderLayer(inFeatCount, n_heads, n_hidden, dropout)

    self.encoder = TransformerEncoder(encoder_layer, num_encoder_layers)

    self.decoder = nn.Linear(inFeatCount, outFeatCount)

  def init_weights(self):
      initrange = 0.1
      self.encoder.weight.data.uniform_(-initrange, initrange)
      self.decoder.bias.data.zero_()
      self.decoder.weight.data.uniform_(-initrange, initrange)

  def forward(self, data):
      encoded = self.encoder(data)
      output = self.decoder(encoded)
      return torch.nn.functional.softmax(output)
  
def predict(model, inputArr):
    inputArr = inputArr.to(device).float()
    
    output = model(inputArr)
    
    return output


In [32]:
max_frames_count = 20
pos_encoder_size = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [105]:
#generate embedding
embedding_arr = gen_loc_list(max_frames_count, pos_encoder_size)

#generate embedded data
data = gen_data(xTrain[0], embedding_arr)

#generate model
model = TransformerModel(15, 5)


criterion = nn.BCELoss()

print(predict(model, data)[:,0])
print(criterion(predict(model, data)[:,0], torch.tensor(yTrain[0])))


tensor([0.3094, 0.3626, 0.5688, 0.3559, 0.5062, 0.1966, 0.3175, 0.7242, 0.4109,
        0.3316], grad_fn=<SelectBackward0>)


/var/folders/66/pn8653k51_75x5ksbpsy8hm80000gn/T/ipykernel_3536/6805551.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return torch.nn.functional.softmax(output)


ValueError: Using a target size (torch.Size([])) that is different to the input size (torch.Size([10])) is deprecated. Please ensure they have the same size.

In [99]:
# training
import torch.optim as optim

def train(model, xTrain, yTrain, xTest, yTest, epochs = 100, lr = 0.001, batch_size = 32):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr = lr)
    
    train_losses = []
    test_losses = []
    train_acc = []
    test_acc = []
    
    model.train()
    optimizer.zero_grad()

    result = predict(model, gen_data(xTrain[0], embedding_arr))
    loss = criterion(result[0,:], torch.tensor(yTrain[0]))
    #loss.backward()
    optimizer.step()

    train_losses.append(loss.item())

    model.eval()
    
        
    
    return train_losses, test_losses, train_acc, test_acc

train(model, xTrain, yTrain, xTest, yTest)

/var/folders/66/pn8653k51_75x5ksbpsy8hm80000gn/T/ipykernel_3536/6805551.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return torch.nn.functional.softmax(output)


ValueError: Using a target size (torch.Size([])) that is different to the input size (torch.Size([2])) is deprecated. Please ensure they have the same size.